# <center><a href='https://notebook.basthon.fr/ocaml/?from=https://raw.githubusercontent.com/mp2i-fsm/mp2i-2021/main/11_tp_additional/module/module.ipynb'>Introduction aux modules OCaml <img src=https://framagit.org/uploads/-/system/project/avatar/55763/basthon_shadow.png width=100></a></center>

Ce TP est une introduction à l'utilisation de [**modules**](https://ocaml.org/docs/modules) en OCaml. Un module est une façon de regrouper plusieurs fonctions (et types) similaires).  
C'est hors-programme mais cela peut quand même être intéressant pour mieux architecturer le code de votre TIPE ou pour mieux comprendre la libraire standard OCaml (qui utilise des modules). Par exemple, `List` est un module.

**Remarque** : Un module est similaire à une classe en Python ou C++, sauf qu'il n'est pas possible d'instancier un module.

## Définition de module

Voici un exemple de module, permettant de faire des calculs sur les polynômes. On stocke ici les coefficients d'un polynôme dans une liste de flottants (par exemple, `[1.; 5.; 0; 2.]` représente le polynôme $1 + 5X + 2X^3$).

In [1]:
module Polynome = struct (* Polynome est un module *)
    let deg p = (* deg p renvoie le degré du polynôme représenté par la liste p *)
        1 + List.length p
    let rec add p1 p2 = match p1, p2 with (* add renvoie la liste représentant la somme de deux polynômes *)
        | [], _ -> p2
        | _, [] -> p1
        | c1::q1, c2::q2 -> (c1 +. c2)::add q1 q2
end

module Polynome :
  sig
    val deg : 'a list -> int
    val add : float list -> float list -> float list
  end


Exemple d'utilisation de ce module :

In [2]:
Polynome.add [1.; 2.; 3.] [0.; 0.; 1.; 2.]

- : float list = [1.; 2.; 4.; 2.]


**Exercice** : Ajouter une fonction `mult` à `Polynom` pour multiplier deux polynômes. Tester.  
**Remarque** : On demande de le faire en O($n^2$) où $n$ est le plus grand degré, mais il existe des méthodes plus rapides (comme [Karastuba](https://fr.wikipedia.org/wiki/Algorithme_de_Karatsuba) ou [FFT](https://fr.wikipedia.org/wiki/Transformation_de_Fourier_rapide)).

## Signature de module

De la même façon que l'on peut définir le type d'une fonction, on peut définir le **type d'un module**. Voici par exemple un type de module pour définir un anneau :

In [3]:
module type Anneau = sig
    type t (* type des éléments du groupe *)
    val neutre_somme : t (* élément neutre pour + *)
    val neutre_mult : t (* élément neutre pour * *)
    val somme : t -> t -> t (* + *)
    val mult : t -> t -> t (* * *)
    val somme_inv : t -> t (* inverse additif *)
    val print : t -> unit (* pour afficher une valeur *)
end

module type Anneau =
  sig
    type t
    val neutre_somme : t
    val neutre_mult : t
    val somme : t -> t -> t
    val mult : t -> t -> t
    val somme_inv : t -> t
    val print : t -> unit
  end


Par exemple, voici la définition de $\mathbb{Z}/5\mathbb{Z}$ sous forme d'un module de type `Anneau` :

In [4]:
module Z5Z : Anneau = struct
    type t = int
    let neutre_somme = 0
    let neutre_mult = 1
    let somme n p = (n + p) mod 5
    let mult n p = (n*p) mod 5
    let somme_inv n = (-n) mod 5
    let print n = Format.printf "%d@." n
end

module Z5Z : Anneau


Exemple d'utilisation :

In [5]:
let e = Z5Z.neutre_mult in
let e2 = Z5Z.somme e e in
Z5Z.print (Z5Z.mult e2 e2)

4


- : unit = ()


**Exercice** : Définir un module de type `Anneau` représentant le corps (donc aussi un anneau) $\mathbb{C}$ des nombres complexes.

## Foncteur

Cette partie est un complément qui peut être passé en première lecture.  
Il est possible de définir des **modules paramétrés (foncteur)**, c'est-à-dire pouvoir définir automatiquement un nouveau module en faisant varier un paramètre. Par exemple, on pourrait définir un foncteur pour obtenir $\mathbb{Z}/n\mathbb{Z}$, pour $n$ quelconque :

In [6]:
module type Integer = sig val n : int end (* type de module utilisé en paramètre de ZnZ *)

module ZnZ (N : Integer) : Anneau = struct
    type t = int
    let neutre_somme = 0
    let neutre_mult = 1
    let somme p q = (p + q) mod N.n
    let mult p q = (p*q) mod N.n
    let somme_inv p = (p) mod N.n
    let print n = Format.printf "%d@." n (* pour afficher l'entier stocké *)
end

module type Integer = sig val n : int end


module ZnZ : functor (N : Integer) -> Anneau


On peut alors définir $\mathbb{Z}/2\mathbb{Z}$ simplement :

In [7]:
module Z2Z = ZnZ (struct let n = 2 end);;

let e = Z2Z.neutre_mult in
Z2Z.print (Z2Z.somme e e)

module Z2Z : Anneau


0


- : unit = ()


## File de priorité par arbre binaire de recherche

On définit le type suivant de module de file de priorité persistante :

In [8]:
module type PrioQ = sig
  type 'a t (* structure contenant les éléments de la file de priorité *)
  val empty : 'a t (* file de priorité vide *)
  val is_empty : 'a t -> bool (* teste si une file de priorité est vide *)
  val add : 'a -> 'a t -> 'a t (* ajoute un élément à la file de priorité *)
  val peek_min : 'a t -> 'a (* renvoie le minimum de la file de priorité *)
  val take_min : 'a t -> 'a t (* extrait le minimum de la file de priorité *)
end

module type PrioQ =
  sig
    type 'a t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val add : 'a -> 'a t -> 'a t
    val peek_min : 'a t -> 'a
    val take_min : 'a t -> 'a t
  end


Comme il s'agit d'une file de priorité persistante, `take_min` et `add` renvoient une nouvelle version de la file de priorité.

**Exercice** : Compléter le module suivant pour implémenter une file de priorité par arbre binaire de recherche.

In [9]:
module Bst : PrioQ = struct
  type 'a t = E | N of 'a * 'a t * 'a t (* type d'arbre binaire *)
  let empty = E
  (* définir is_empty, add, peek_min, take_min *)
end

error: compile_error

## Algorithme de Dijkstra

**Exercice** : Écrire une fonction implémentant l'algorithme de Dijkstra en utilisant un `Bst` comme file de priorité. On pourra adapter le code du cours sur l'algorithme de Dijkstra. Tester votre fonction.

## Prolongements

**Exercice** : Réécrire l'algorithme de Dijkstra sous forme d'un foncteur, de façon à être utilisable avec n'importe quelle `PrioQ`. Tester. On pourra compléter le code suivant :

In [10]:
module Dijkstra (Q : PrioQ) = struct
    let distances g r = (* renvoie un tableau des distances de r aux sommets de g *)
        (* compléter *)
end

error: compile_error

**Exercice** : Écrire un module de type `PrioQ` implémenté par un tas. Vérifier que `Dijkstra` fonctionne avec ce nouveau `PrioQ`, avec des modifications minimes.

**Exercice** : Écrire un type de module `Graph` (avec des fonctions `add_edge`, `is_edge`... et réimplémenter l'algorithme de Dijkstra sous forme d'un foncteur paramétré à la fois par un `Graph` et une `PrioQ`.